In [1]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [2]:
import requests
import time
summaries = {}

# API Setup
api_url = "https://api.groq.com/openai/v1/chat/completions"
api_key = os.getenv("GROQ_API_KEY")

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

In [3]:
# Prompting Techniques

prompt_templates = {
    "Chain-of-Thought": """
Let's analyze this step-by-step:
1. Identify the background of the research.
2. Understand the research objective.
3. Describe the methodology.
4. Summarize the results obtained.
5. Extract the key takeaways.
Now, summarize this article accordingly:
[Insert Article Text Here]
""",

    "Tree-of-Thought": """
Break down the research article into branches:
- Background
- Objective
- Method
- Results
- Takeaways
Explore each branch and summarize concisely:
[Insert Article Text Here]
""",

    "Role-based Prompting": """
Assume the role of a medical science journalist. Your job is to summarize complex research into a single, digestible paragraph. Include background, objective, method, result, and takeaways.
Research:
[Insert Article Text Here]
""",

    "ReAct Prompting": """
Think step-by-step and act:
Thought: What is the paper about?
Action: Summarize background and objective.
Thought: What was done in the study?
Action: Summarize methodology and results.
Thought: What should a reader remember?
Action: Write the key takeaways.
Begin with the article:
[Insert Article Text Here]
""",

    "Directional Stimulus Prompting": """
Focus on a detailed yet concise paragraph summary that includes:
- Why the research was done
- What was done
- What was found
- Why it matters
Keep it within 150 words:
[Insert Article Text Here]
""",

    "Step-Back Prompting": """
First step back and consider what a general audience needs to understand:
Now, write a paragraph that explains the research in simple terms with background, aim, methods, results, and significance:
[Insert Article Text Here]
"""
}

In [4]:
# Running all prompting techniques
def summarize_article(article_text):
  for name, prompt in prompt_templates.items():
    print(f"\nSummary using {name} prompting technique:\n" + "-"*60)
    prompt_filled = prompt.replace("[Insert Article Text Here]", article_text)

    messages = [
        {"role": "system", "content": "You are an expert summarizer specialized in medical research."},
        {"role": "user", "content": prompt_filled}
    ]

    response = requests.post(api_url, headers=headers, json={
        "model": "llama3-8b-8192",
        "messages": messages,
        "temperature": 0.3,
        "max_tokens": 300
    })

    if response.status_code == 200:
        summary = response.json()['choices'][0]['message']['content']
        print(summary)
    else:
        print(f"Error {response.status_code}: {response.text}")
    summaries[name] = summary
    time.sleep(10)

In [5]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]

with open(filename, "r", encoding="utf-8") as f:
    article_text = f.read()
summarize_article(article_text)

Saving Advanced_Prompting_Article_Summarization.txt to Advanced_Prompting_Article_Summarization.txt

Summary using Chain-of-Thought prompting technique:
------------------------------------------------------------
Here's a summary of the article:

**Background:** Cancer is a significant public health concern, with millions of new cases and deaths reported annually. Early detection is crucial for improving patient outcomes, but current screening and diagnostic practices often struggle with sensitivity, especially in under-resourced regions.

**Objective:** The study aims to design, implement, and evaluate a machine learning-based framework for early cancer detection using non-invasive and routinely collected patient data.

**Methodology:** The researchers compiled a retrospective dataset of 45,000 patients from five major hospitals in three countries, including clinical parameters such as blood test results, demographic information, and low-resolution imaging features. They trained and 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
with open("/content/drive/MyDrive/Prompting_Summaries.txt", "w") as f:
    for name, content in summaries.items():
        f.write(f"=== {name} ===\n{content}\n\n")